In [2]:
import pandas as pd
import os
import numpy as np
import glob
import matplotlib.pyplot as plt

%matplotlib inline

# IFLS

In [4]:
a=pd.read_stata("b3a_cov.dta")
b=pd.read_stata("b3a_tk1.dta")
c=pd.read_stata("b3a_tk2.dta")

a.set_index(["hhid07_9", "pid07"], inplace=True)
b.set_index(["hhid07_9", "pid07"], inplace=True)
c.set_index(["hhid07_9", "pid07"], inplace=True)

In [11]:
pd.read_stata("b3a_tk2.dta", iterator=True).variable_labels()

{'hhid07_9': 'Household ID number',
 'pid07': "Respondent's HH member ID #",
 'tk18abx': 'What is telephone number of the company?',
 'tk20a': 'What are your main duties: Job 1?',
 'tk20aax': 'Do you know how many workers',
 'tk20ab': 'Is it......',
 'tk21a': 'Total # of hrs worked last/wk: Job 1',
 'tk22a': 'Normal total hrs workd per week: Job 1',
 'tk23a': 'Number of weeks worked per year',
 'tk23a2y': 'How long at this job (years)',
 'tk23a2m': 'How long at this job (months)',
 'tk23a4': 'Member of labor union / bus association?',
 'tk24a': 'What is your working status: Job 1?',
 'tk24a1id': 'PID',
 'tk24a2a': 'How did you get this job?',
 'tk24a2': 'By what system were you paid last month?',
 'tk24a5': 'Do you work with a contract',
 'tk24a6x': 'Do you know the term of your contract',
 'tk24a61': 'What is the term of your contract(month)',
 'tk24a62': 'What is the term of your contract(year)',
 'tk24a7m': 'Month the current contract started',
 'tk24a7y': 'Year the current contract

In [5]:
z = pd.concat([a, b, c], axis=1)
# our interest just people in youth age (15-24 based on ILO standard)
z = z[z["age"] < 25]

In [6]:
z["tk01"].value_counts(dropna=False)

1.0     2720
3.0     2104
4.0     1627
95.0     310
2.0      288
NaN      218
7.0       31
5.0        2
Name: tk01, dtype: int64

In [8]:
z.columns

Index(['idivwr', 'editor', 'respndnt', 'panel', 'age', 'marstat', 'sex',
       'dob_day', 'dob_mth', 'dob_yr',
       ...
       'tk23b2mx', 'tk23b2yx', 'tk20ba', 'tk25b1', 'tk25b2', 'tk25b2b',
       'tk26b1', 'tk26b3', 'version', 'module'],
      dtype='object', length=230)

**We ignore the missing values of tk01 column (main activity last week) because relatively small**

### Def Training 1st ( tk25a9x ) -- Last year got training?

In [21]:
z["tk25a8"].unique()

array([ 1., nan,  3.,  9.])

In [10]:
# without imputation
pd.crosstab(z["tk01"], z["tk25a9x"], margins=True)

tk25a9x,1.0,2.0,3.0,All
tk01,,,,
1.0,191,90,93,374
2.0,0,1,0,1
3.0,6,6,2,14
4.0,3,5,3,11
7.0,0,0,2,2
95.0,1,0,0,1
All,201,102,100,403


In [13]:
one = z.copy()
one = one[["tk01", "tk25a9x"]].dropna()
neet1 = one[((one["tk25a9x"] == 3.)) & ((one["tk01"] == 2.) | (one["tk01"] == 2.) | (one["tk01"] == 7.) | (one["tk01"] == 95.))]
eet1 = one[~(((one["tk25a9x"] == 3.)) & ((one["tk01"] == 2.) | (one["tk01"] == 2.) | (one["tk01"] == 7.) | (one["tk01"] == 95.)))]
eet1

,,tk01,tk25a9x
hhid07_9,pid07,,
001220000,6.0,1.0,2.0
002250000,4.0,1.0,2.0
003050000,6.0,1.0,1.0
003260003,1.0,1.0,1.0
004100000,9.0,1.0,1.0
...,...,...,...
320080000,3.0,1.0,1.0
320100000,5.0,1.0,1.0
320180000,5.0,1.0,1.0


In [15]:
neets1 = (5 )/ 403
print ("Based on IFLS (scheme 1), the propotion of NEETs in Indonesia equal to" , round(neets1 * 100, 2) , "% -(n=403)")

Based on IFLS (scheme 1), the propotion of NEETs in Indonesia equal to 1.24 % -(n=403)


In [16]:
# with imputation
imp = z["tk25a9x"].fillna(3.)
pd.crosstab(z["tk01"], imp, margins=True)

tk25a9x,1.0,2.0,3.0,All
tk01,,,,
1.0,191,90,2439,2720
2.0,0,1,287,288
3.0,6,6,2092,2104
4.0,3,5,1619,1627
5.0,0,0,2,2
7.0,0,0,31,31
95.0,1,0,309,310
All,201,102,6779,7082


In [15]:
two = z.copy()
two = two[["tk01", "tk25a9x"]]
two["tk25a9x"] = two["tk25a9x"].fillna("3:Not receive any training in the last 12 months").astype("str")
neet2 = two[((two["tk25a9x"] == "3:Not receive any training in the last 12 months")) & ((two["tk01"] == "2:Job searching") | (two["tk01"] == "4:Housekeeping") | (two["tk01"] == "7:Sick/disable") | (two["tk01"] == "95:Other"))]
eet2 = two[~(((two["tk25a9x"] == "3:Not receive any training in the last 12 months")) & ((two["tk01"] == "2:Job searching") | (two["tk01"] == "4:Housekeeping") | (two["tk01"] == "7:Sick/disable") | (two["tk01"] == "95:Other")))]
eet2

tk01  \
hhid14_9  pid14                                                      
001060000 8.0                                   3:Attending school   
001220009 1.0                                   3:Attending school   
001224100 3.0                                   3:Attending school   
001224200 1.0                                   3:Attending school   
001240000 12.0   1:Working/trying to get work/helping to earn i...   
...                                                            ...   
321280000 12.0   1:Working/trying to get work/helping to earn i...   
321290000 6.0                                   3:Attending school   
          7.0    1:Working/trying to get work/helping to earn i...   
321291100 3.0                                   3:Attending school   
          4.0                                   3:Attending school   

                                                          tk25a9x  
hhid14_9  pid14                                                    
001060000 8.0    3:Not receive any training in the last 12 months  
001220009 1.0    3:Not receive any training in the last 12 months  
001224100 3.0    3:Not receive any training in the last 12 months  
001224200 1.0                                  2:Less than a week  
001240000 12.0   3:Not receive any training in the last 12 months  
...                                                           ...  
321280000 12.0   3:Not receive any training in the last 12 months  
321290000 6.0    3:Not receive any training in the last 12 months  
          7.0    3:Not receive any training in the last 12 months  
321291100 3.0    3:Not receive any training in the last 12 months  
          4.0    3:Not receive any training in the last 12 months  

[5657 rows x 2 columns]

In [17]:
neets2 = (287+ 1619 + 2 + 31 + 309)/ 7082
print ("Based on IFLS (scheme 2), the propotion of NEETs in Indonesia equal to" , round(neets2 * 100, 2) , "% -(n=7082)")

Based on IFLS (scheme 2), the propotion of NEETs in Indonesia equal to 31.74 % -(n=7082)


### Def Training 1st ( tk25a8 ) -- ever got training?

In [18]:
#without imputation
pd.crosstab(z["tk01"], z["tk25a8"], margins=True)

tk25a8,1.0,3.0,9.0,All
tk01,,,,
1.0,374,1177,1,1552
2.0,1,10,0,11
3.0,14,36,0,50
4.0,11,52,0,63
7.0,2,4,0,6
95.0,1,15,0,16
All,403,1294,1,1698


In [19]:
three = z.copy()
three = three[["tk01", "tk25a8"]].dropna()
neet3 = three[((three["tk25a8"] == 3.)) & ((three["tk01"] == 2.) | (three["tk01"] == 4.) | (three["tk01"] == 7.) | (three["tk01"] == 95.))]
eet3 = three[~(((three["tk25a8"] == 3.)) & ((three["tk01"] == 2.) | (three["tk01"] == 4.) | (three["tk01"] == 7.) | (three["tk01"] == 95.)))]
eet3

,,tk01,tk25a8
hhid07_9,pid07,,
001220000,6.0,1.0,1.0
002250000,4.0,1.0,1.0
002290004,1.0,1.0,3.0
003050000,6.0,1.0,1.0
003090000,14.0,1.0,3.0
...,...,...,...
321173100,4.0,1.0,3.0
321200000,3.0,1.0,1.0
321260000,6.0,1.0,1.0


In [20]:
neets3 = (10 + 52 + 4 + 15)/ 1698
print ("Based on IFLS (scheme 3), the propotion of NEETs in Indonesia equal to" , round(neets3 * 100, 2) , "% -(n=1698)")

Based on IFLS (scheme 3), the propotion of NEETs in Indonesia equal to 4.77 % -(n=1698)


In [22]:
#imputation with all no
io = z["tk25a8"].fillna(3.)
pd.crosstab(z["tk01"], io, margins=True)

tk25a8,1.0,3.0,9.0,All
tk01,,,,
1.0,374,2345,1,2720
2.0,1,287,0,288
3.0,14,2090,0,2104
4.0,11,1616,0,1627
5.0,0,2,0,2
7.0,2,29,0,31
95.0,1,309,0,310
All,403,6678,1,7082


In [23]:
four = z.copy()
four = four[["tk01", "tk25a8"]]
four["tk25a8"] = four["tk25a8"].fillna(3.)
neet4 = four[((four["tk25a8"] == 3.)) & ((four["tk01"] == 2.) | (four["tk01"] == 4.) | (four["tk01"] == 7.) | (four["tk01"] == 95.))]
eet4 = four[~(((four["tk25a8"] == 3.)) & ((four["tk01"] == 2.) | (four["tk01"] == 4.) | (four["tk01"] == 7.) | (four["tk01"] == 95.)))]
eet4

tk01  tk25a8
hhid07_9  pid07              
001220000 6.0     1.0     1.0
          7.0     1.0     3.0
          9.0     3.0     3.0
001220008 1.0     3.0     3.0
002030000 3.0     NaN     3.0
...               ...     ...
321270000 6.0     1.0     3.0
          7.0     3.0     3.0
321280000 9.0     1.0     3.0
          10.0    1.0     3.0
321300000 3.0     3.0     3.0

[5059 rows x 2 columns]

In [26]:
neets4 = (287 + 1616 + 2 + 29 + 309)/ 7082
print ("Based on IFLS (scheme 3), the propotion of NEETs in Indonesia equal to" , round(neets4 * 100, 2) , "% -(n=7082)")

Based on IFLS (scheme 3), the propotion of NEETs in Indonesia equal to 31.67 % -(n=7082)


In [6]:
287 + 1616 + 2 + 29 + 309 - 7082

-4839